In [5]:
import pandas as pd
import requests
import scipy.signal
import numpy as np

from datetime import datetime
from dateutil import parser

from urllib import parse
from data_pull import data_pull_date
from data_pull import data_pull_all

In [2]:
# System Frequency (Dec 2019)
# f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9    
# https://data.nationalgrideso.com/system/system-frequency-data/r/december_2019_-_historic_frequency_data

# "9bc4746e-3152-4c6f-886e-58377ab88e0e" # Nov 2021

### Load Data via API or from CSV

In [4]:
path = "C:\\Users\\kmp57\\Desktop\\Documents\\GitHub\\grid-uk-ml\\data\\"

## datacode = "f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9"
## entry_limit = '1300000'

## freq = data_pull_all(datacode,entry_limit)
freq_names = 

freq = pd.read_csv(path + 'RawFrequencyData\f 2019 12.csv')
freq['dtm'] = pd.to_datetime(freq['dtm'], format="%Y/%m/%d %H:%M:%S")

start_time = freq.loc[0,"dtm"]
start_str = "Start Time: {}".format(start_time)
print(start_str)

idx = freq.index[-1]
end_time = freq.loc[idx,"dtm"]
end_str = "End Time: {}".format(end_time)
print(end_str)

period = freq.loc[1,"dtm"] - start_time
period_str = "Period: {}".format(period)
print(period_str)  
print(len(freq))
freq.dtypes

FileNotFoundError: [Errno 2] File b'C:\\Users\\kmp57\\Desktop\\Documents\\GitHub\\grid-uk-ml\\data\\RawFrequencyData\x0c 2019 12.csv' does not exist: b'C:\\Users\\kmp57\\Desktop\\Documents\\GitHub\\grid-uk-ml\\data\\RawFrequencyData\x0c 2019 12.csv'

### Rearrange Data by Date

In [ ]:
dates = freq.dtm.dt.date[0::86400]
 
freq_date = pd.DataFrame()
for ii in dates:
    date_iter = ii.strftime('%Y-%m-%d')
    foo = freq.loc[freq.dtm.dt.date==ii,'f'].reset_index(drop=True)
    freq_date[date_iter] = foo
    
    print(date_iter)
#     print(ii)
#     print(foo)



### Calculate PSD for each date every 30 min and estimate noise power in different frequency bands


In [ ]:
fs = 1    # Sampling Freq (Hz)
T = 30*60 # Measurement time (s)

idx_list = freq_date.index[0::T] # Find index for every 30 min

band_edges = np.linspace(0,0.5,6) # Define Frequency Bands
band_edges = np.round(band_edges, 1)

band_names = list_string = map(str, band_edges[1:])
band_names = list(band_names)

col_labels = ["dtm"] +  band_names

data = []
for date in dates:
    date_iter = date.strftime('%Y-%m-%d')
    for ii in range(len(idx_list)):
        psd_band = []

        # Calculate PSD for each 30 min
        start_idx = idx_list[ii]
        end_idx = idx_list[ii] + T   

        
        signal = freq_date.loc[start_idx:end_idx,date_iter] - 50  # subtract 50 so signal is centred at 0Hz
        (ff, psd) = scipy.signal.periodogram(signal, fs, scaling='density')
        ## Frequency labelled ff as it is frequency of frequency measurements
        
        
        # Label each 30 with datetime
        dtm_str =  date_iter + datetime.fromtimestamp(start_idx).strftime("%H:%M:%S")
        dtm = datetime.strptime(dtm_str,"%Y-%m-%d%H:%M:%S")
    
        # Calculate average PSD within each band
        for jj in range(len(band_edges)-1):
            band_start = band_edges[jj]
            band_end = band_edges[jj+1]
            psd_band.append( psd[(ff >= band_start) & (ff < band_end)].mean() )           
            
        data.append( [dtm] + psd_band ) 

# Build Datetimes and Averages into Dataframe
freq_psd = pd.DataFrame (data, columns = col_labels)

### Save Data 

In [ ]:
filename = freq_psd.loc[0,"dtm"]
filename = "Data\\Frequency\\psd_" + filename.strftime("%Y-%m") + ".csv"
print(filename)
freq_psd.to_csv(filename)
